# 1. Start Isaac

In [1]:
import sys
import os
import signal

from isaacsim import SimulationApp

CONFIG = {
  'headless': True,
  'display_options': 3286
}

# Example for creating a RTX lidar sensor and publishing PointCloud2 data
simulation_app = SimulationApp(CONFIG)
simulation_app.update()

spinning = True
def sigint_handler(sig, frame):
  global spinning
  spinning = False
signal.signal(signal.SIGINT, sigint_handler)

kernel = get_ipython().kernel
kernel.pre_handler_hook = lambda: None
kernel.post_handler_hook = lambda: None

def spin(update_function, *args, **kwargs):
  global spinning
  print('gui can be used now. interrupt kernel to go back to notebook')
  while spinning:
    update_function(*args, **kwargs)
  spinning = True

def resume_gui():
  spin(lambda: simulation_app.update())
  
if CONFIG['headless']:
  del spin
  del resume_gui
  

print('workdir:', os.getcwd())


Starting kit application with the following args:  ['/f/ov/pkg/isaac-sim-4.1.0/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/f/ov/pkg/isaac-sim-4.1.0/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/f/ov/pkg/isaac-sim-4.1.0/kit', '--/persistent/app/viewport/displayOptions=3286', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True', '--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--/app/fastShutdown=False', '--ext-folder', '/f/ov/pkg/isaac-sim-4.1.0/exts', '--ext-folder', '/f/ov/pkg/isaac-sim-4.1.0/apps', '--/physics/cudaDevice=0', '--portable', '--no-window', '--/app/window/hideUi=1']
Passing the following args to the base kit application:  ['-f', '/home/linh/.local/share/jupyter/runtime/kernel-7239753a-3294-47e6-8a88-feab89dba6dc.json']
[Warning] [omni.isaac.kit.simulation_app] f

Warp 1.2.1 initialized:
   CUDA Toolkit 11.8, Driver 12.5
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA GeForce RTX 4060" (8 GiB, sm_89, mempool enabled)
   Kernel cache:
     /home/linh/.cache/warp/1.2.1
startup
[2.472s] [ext: omni.kit.window.property-1.11.1] startup
[2.474s] [ext: omni.usdphysics-106.0.20] startup
[2.479s] [ext: omni.graph.tools-1.78.0] startup
[2.502s] [ext: omni.kit.viewport.utility-1.0.17] startup
[2.503s] [ext: omni.kit.property.usd-3.21.28] startup
[2.513s] [ext: omni.graph-1.135.0] startup
[2.641s] [ext: omni.graph.image.core-0.3.2] startup
[2.648s] [ext: omni.graph.ui-1.70.0] startup
[2.671s] [ext: omni.graph.action_core-1.1.4] startup
[2.686s] [ext: omni.kit.window.cursor-1.1.2] startup
[2.688s] [ext: omni.graph.image.nodes-1.0.2] startup
[2.691s] [ext: omni.graph.action_nodes-1.23.0] startup
[2.701s] [ext: omni.kit.viewport.menubar.core-106.0.2] startup
[2.731s] [ext: omni.graph.nodes-1.143.0] startup
[2.754s] [ext: omni.graph.action-1.102

In [2]:

import omni
from omni.isaac.core import World
from omni.isaac.core.utils.stage import *
from omni.isaac.core.objects import *
from omni.isaac.core.utils.rotations import *
from omni.isaac.core.utils.prims import *
from omni.isaac.core.utils.xforms import *
from omni.isaac.sensor import *
from omni.isaac.nucleus import *

from dimenvue import Dimenvue

# 3. Setup scene

In [3]:
def setup_sim():
    create_new_stage()    
    world = World(physics_dt=1.0/400, rendering_dt=1.0/10)
    asset_root = get_assets_root_path()
    office_uds_path = asset_root + "/Isaac/Environments/Office/office.usd"
    assert is_file(office_uds_path), "Office environment not found"
    add_reference_to_stage(office_uds_path, prim_path="/World/office")
    world.render()
    light = create_prim('/World/light', 'RectLight', translation=[0,0,2.97])
    light.GetAttribute('inputs:intensity').Set(1000.0)
    light.GetAttribute('inputs:height').Set(150.0)
    light.GetAttribute('inputs:width').Set(100.0)
    dimenvue = Dimenvue(
        config_filfe_name="dimenvue_config.json",
        prim_path="/world/dimenvue",
        name="dimenvue"    
    )
    world.scene.add(dimenvue)
    world.reset(soft=False)
    return world, dimenvue



# 4. Run sim and log data

In [4]:
from PIL import Image
import numpy as np
import io
import pandas as pd
from data_logger import DataLogger


def trajectory_generator(path: str, preview=False):
    """
    in case the trajectory is splitted into multiple files, this generator can be used to iterate over all of them
    """
    trajectory = pd.read_csv(path)
    if preview:
        trajectory = trajectory.iloc[:500]
    for i in range(len(trajectory)):
        yield trajectory.iloc[i]
    
def unpack_point(point):
    position = point[['x', 'y', 'z']].to_numpy()
    orientation = point[['qw', 'qx', 'qy', 'qz']].to_numpy()
    return position, orientation

def test():
    world, dimenvue = setup_sim()
        
    trajectory = trajectory_generator('./data/trajectory/office2/400/0.csv', preview=False)
    path_iter = iter(trajectory)

    dimenvue.set_local_pose(*unpack_point(next(path_iter)))
    
    running = True
    def path_advance(event):
        nonlocal running
        try:
            dimenvue.set_local_pose(*unpack_point(next(path_iter)))            
        except StopIteration:
            running = False
    
    world.add_physics_callback("path_advance", path_advance)
    
    def compress(data_frame):
        for cam in data_frame['cameras']:
            data_frame['cameras'][cam]['compressed'] = []
            for frame in data_frame['cameras'][cam]['rgba']:
                if np.prod(frame.shape) == 0:
                    data_frame['cameras'][cam]['compressed'].append(None)
                    continue
                rgba_image = Image.fromarray(frame, 'RGBA')
                buffer = io.BytesIO()
                rgba_image.save(buffer, format="PNG")
                png_bytes = buffer.getvalue()
                buffer.close()
                data_frame['cameras'][cam]['compressed'].append(png_bytes)
            del data_frame['cameras'][cam]['rgba']
        return data_frame
    
    logger = DataLogger('office_sim', chunk_size_mb=500)
    # doesn't really help. using tar on the uncompressed data is easier
    # logger.set_transform(compress)
    
    world.reset(soft=True)
    while running:
        world.step()
        current_data = dimenvue.consume_frame()
        logger.log(current_data)


test()

skip empty frame
[9.645s] Simulation App Startup Complete


2024-12-21 07:28:09 [42,330ms] [Error] [omni.physx.plugin] PhysX error: PxRigidDynamic::setLinearVelocity: Body must be non-kinematic!, FILE /builds/omniverse/physics/physx/source/physx/src/NpRigidDynamic.cpp, LINE 232
2024-12-21 07:28:09 [42,330ms] [Error] [omni.physx.plugin] PhysX error: PxRigidDynamic::setAngularVelocity: Body must be non-kinematic!, FILE /builds/omniverse/physics/physx/source/physx/src/NpRigidDynamic.cpp, LINE 256
2024-12-21 07:28:09 [42,647ms] [Error] [omni.physx.plugin] PhysX error: PxRigidDynamic::setLinearVelocity: Body must be non-kinematic!, FILE /builds/omniverse/physics/physx/source/physx/src/NpRigidDynamic.cpp, LINE 232
2024-12-21 07:28:09 [42,647ms] [Error] [omni.physx.plugin] PhysX error: PxRigidDynamic::setAngularVelocity: Body must be non-kinematic!, FILE /builds/omniverse/physics/physx/source/physx/src/NpRigidDynamic.cpp, LINE 256


skip empty frame


# 5. Visualize and debug

In [1]:
import sys
import os

import numpy as np
import pandas as pd
import rerun as rr

from matplotlib import pyplot as plt

from rotations import *

rr.init('office_trajectory', spawn=True)


def Axes(length=1.0):
    return rr.Arrows3D(
        vectors=[[length, 0, 0], [0, length, 0], [0, 0, length]],
        colors=[[255,0,0], [0,255,0], [0,0,255]],
    )

rr.log('/world', Axes(0.1), static=True)


[2024-12-10T01:29:54Z INFO  re_sdk_comms::server] Hosting a SDK server over TCP at 0.0.0.0:9876. Connect with the Rerun logging SDK.
[2024-12-10T01:29:54Z INFO  winit::platform_impl::platform::x11::window] Guessed window scale factor: 1
[2024-12-10T01:29:54Z INFO  re_sdk_comms::server] New SDK client connected from: 127.0.0.1:40144


In [2]:
import json
import sys
import os

import numpy as np
import pandas as pd
import rerun as rr

from matplotlib import pyplot as plt

from rotations import *

rr.init('office_trajectory', spawn=True)


def Axes(length=1.0):
    return rr.Arrows3D(
        vectors=[[length, 0, 0], [0, length, 0], [0, 0, length]],
        colors=[[255,0,0], [0,255,0], [0,0,255]],
    )

rr.log('/world', Axes(0.1), static=True)

def rr_transform(pos:np.ndarray, angles:np.ndarray):
    wxyz = euler_angles_to_quat(angles, degrees=True, extrinsic=True)
    xyzw = np.roll(wxyz, -1)
    return rr.Transform3D(
        translation=pos,
        quaternion=xyzw
    )

def rr_set_up_frame():
    # world, dimenvue = setup_sim()
    # dimenvue_config = dimenvue.config
    dimenvue_config = json.load(open('dimenvue_config.json'))
    rr.set_time_seconds('sim_time', 0.0)
    rr.log('/world', rr.Clear(recursive=True))
    rr.log('/world', Axes(1), static=True)
    c = dimenvue_config['chassis']
    p, r = c['world_chassis_translate'], c['world_chassis_rotate']
    rr.log('/world/chassis', rr_transform(p, r))
    rr.log('/world/chassis', Axes(1))
    if 'lidar' in dimenvue_config:
        c = dimenvue_config['lidar']
        p, r = c['chassis_lidar_translate'], c['chassis_lidar_rotate']
        rr.log('/world/chassis/lidar', rr_transform(p, r))
        rr.log('/world/chassis/lidar', Axes(1))
    if 'cameras' in dimenvue_config:
        c = dimenvue_config['cameras']
        p, r = c['chassis_cameras_translate'], c['chassis_cameras_rotate']
        rr.log('/world/chassis/cameras', rr_transform(p, r))
        for i in range(c['num_cameras']):
            p, r = c['camera_translate'][i], c['camera_rotate'][i]
            rr.log(f'/world/chassis/cameras/camera{i}', rr_transform(p, r))
            rr.log(f'/world/chassis/cameras/camera{i}', Axes(1))
    
            calib = c['calibrate_config']
            w, h = calib['camera_resolution']
            if calib['use_intrinsics']:
                ((fx, _, cx),(_,fy,cy),(_,_,_)) = calib['camera_intrinsics']
                f = (fx+fy)/2
            else:
                h_fov = calib['horizontal_fov_deg']*np.pi/180
                f = w/(2*np.tan(h_fov/2))
                cx, cy = w/2, h/2
                
            rr.log(f'/world/chassis/cameras/camera{i}/image', rr.Pinhole(
                resolution=(w,h),
                camera_xyz=rr.ViewCoordinates.RDF, #same as ROS
                focal_length=(f, f),
                principal_point=(cx, cy),
                image_plane_distance=1.0
            ))

rr_set_up_frame()

In [ ]:
def log_rgb(data):
    rr.set_time_seconds('sim_time', 0)
    

In [ ]:
import pickle

dump_file = './data/trajectory/office/30/lidar_OS1_REV6_32ch10hz512res.pkl'

import pickle

with open(dump_file, 'rb') as f:
    lidar_data = pickle.load(f)

In [ ]:
TF_chassis_lidar = np.eye(4)
TF_chassis_lidar[:3, :3] = euler_to_rot_matrix(dimenvue_config['lidar']['chassis_lidar_rotate'], degrees=True)
TF_chassis_lidar[:3, 3] = dimenvue_config['lidar']['chassis_lidar_translate']


def get_transform(p, q):
    TF_world_chassis = np.eye(4)
    TF_world_chassis[:3, :3] = quat_to_rot_matrix(q)
    TF_world_chassis[:3, 3] = p
    return np.dot(TF_world_chassis, TF_chassis_lidar)

def transform(points, TF):
    if points.size == 0:
        return points
    points_world = np.dot(TF[:3, :3], points.T).T + TF[:3, 3]
    return points_world

In [ ]:
# log lidar data
lidar_data = lidar_data['lidar']
for frame in lidar_data:
    t = frame['rendering_time']
    points = frame['xyz']
    rr.set_time_seconds('sim_time', t)
        
    position = frame['chassis_position']
    orientation_wxyz = frame['chassis_orientation']
    orientation_xyzw = xyzw(orientation_wxyz)
    
    rr.log(f'/world/chassis', rr.Transform3D(translation=position, quaternion=orientation_xyzw))
    rr.log(f'/world/chassis/lidar/points', rr.Points3D(points))
    TF = get_transform(position, orientation_wxyz)
    points_world = transform(points, TF)
    rr.log(f'/world/map/{t}', rr.Points3D(points_world))
